In [5]:
########################################
# Data Loading and Preprocessing
########################################

path = r"/content/drive/MyDrive/Datasets_ALL/transformed_dataset-yrs-22-23.csv"
df = pd.read_csv(path, low_memory=False)

print("Unique values in 'action_taken' before mapping:")
print(df['action_taken'].unique())

print("\nCounts of each 'action_taken' value:")
print(df['action_taken'].value_counts())

# Map action_taken to a binary 'loan_approved' target
loan_approved_mapping = {
    1: 1,  # Loan originated -> Approved
    2: 0,  # Application approved but not accepted -> Denied
    3: 0,  # Application denied -> Denied
    4: 0,  # Application withdrawn -> Denied
    5: 0,  # File closed for incompleteness -> Denied
    6: 1,  # Purchased loan -> Approved
    7: 0,  # Preapproval request denied -> Denied
    8: 0,  # Preapproval request approved but not accepted -> Denied
}

df['loan_approved'] = df['action_taken'].map(loan_approved_mapping)
df_binary = df[df['loan_approved'].notnull()].copy()
df_binary['loan_approved'] = df_binary['loan_approved'].astype(int)
df_binary.drop('action_taken', axis=1, inplace=True)

print("Loan Approved Distribution:")
print(df_binary['loan_approved'].value_counts())

Unique values in 'action_taken' before mapping:
[1 6 2 5 3 8 7 4]

Counts of each 'action_taken' value:
action_taken
1    2097977
3     666726
4     593079
6     460066
5     212104
2     104840
8      14227
7       4299
Name: count, dtype: int64
Loan Approved Distribution:
loan_approved
1    2558043
0    1595275
Name: count, dtype: int64


In [6]:
########################################
# Feature and Target Selection
########################################

feature_cols = [
    'tract_to_msa_income_percentage',
    'ffiec_msa_md_median_family_income',
    'tract_minority_population_percent',
    'interest_rate',
    # One-hot encoded categorical features
    'race_0', 'race_1', 'race_2', 'race_3', 'race_4', 'race_5', 'race_6', 'race_7', 'race_8',
    'gender_0', 'gender_1', 'gender_2', 'gender_3',
    'ethnicity_0', 'ethnicity_1', 'ethnicity_2', 'ethnicity_3', 'ethnicity_4',
    'loan_type_2', 'loan_type_3', 'loan_type_4',
    'loan_purpose_2', 'loan_purpose_4', 'loan_purpose_5', 'loan_purpose_31', 'loan_purpose_32',
    'lien_status_2',
    'construction_method_2',
    'occupancy_type_2', 'occupancy_type_3'
]

X = df_binary[feature_cols]
y = df_binary['loan_approved']

X_train, X_test, y_train, y_test, train_df, test_df = train_test_split(
    X, y, df_binary, test_size=0.2, random_state=42
)

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed")

Training set size: 3322654
Testing set size: 830664
Feature scaling completed


In [7]:
########################################
# Model Building and Training
########################################
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(tf.keras.Input(shape=(X_train_scaled.shape[1],)))
model.add(Dense(units=64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))  # Output layer for binary classification

print(model.summary())
print(f"Number of weights: {len(model.weights)}")

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           2,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,609 (18.00 KB)

 Trainable params: 4,481 (17.50 KB)

 Non-trainable params: 128 (512.00 B)

None
Number of weights: 10
Epoch 1/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 373s 2ms/step - accuracy: 0.7303 - loss: 0.5295 - val_accuracy: 0.8887 - val_loss: 0.3027
Epoch 2/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 377s 2ms/step - accuracy: 0.8505 - loss: 0.3779 - val_accuracy: 0.8892 - val_loss: 0.2902
Epoch 3/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 369s 2ms/step - accuracy: 0.8618 - loss: 0.3622 - val_accuracy: 0.9069 - val_loss: 0.2752
Epoch 4/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 406s 2ms/step - accuracy: 0.8651 - loss: 0.3577 - val_accuracy: 0.9038 - val_loss: 0.2703
Epoch 5/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 444s 2ms/step - accuracy: 0.8670 - loss: 0.3546 - val_accuracy: 0.9055 - val_loss: 0.2693
Epoch 6/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 392s 2ms/step - accuracy: 0.8692 - loss: 0.3514 - val_accuracy: 0.9130 - val_loss: 0.2589
Epoch 7/100
166133/166133 ━━━━━━━━━━━━━━━━━━━━ 442s 2ms/step - accuracy: 0.8709 - loss: 0.3487 - val_accuracy: 0.9056 - val_loss: 0.2689
Epoch 8/100
16

In [8]:
########################################
# Prediction and Evaluation
########################################

y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob >= 0.5).astype(int).reshape(-1)
print(y_test.shape)
print(y_pred.shape)

def evaluate_model(y_true, y_pred, model_name):
    print(f"\n### {model_name} Performance ###")
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (TPR): {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")

    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2, 2):
        cm_df = pd.DataFrame(cm, index=['Actual Negative', 'Actual Positive'],
                             columns=['Predicted Negative', 'Predicted Positive'])
    elif cm.shape == (1, 1):
        if y_true.iloc[0] == 0:
            cm_df = pd.DataFrame(cm, index=['Actual Negative'], columns=['Predicted Negative'])
        else:
            cm_df = pd.DataFrame(cm, index=['Actual Positive'], columns=['Predicted Positive'])
    else:
        cm_df = pd.DataFrame(cm)

    display(cm_df)

evaluate_model(y_test, y_pred, "Neural Network (Keras)")
print("Loan Approved Distribution in Testing Set:")
print(y_test.value_counts())

25959/25959 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step
(830664,)
(830664,)

### Neural Network (Keras) Performance ###
Accuracy: 0.9184
Precision: 0.9547
Recall (TPR): 0.9106
F1-Score: 0.9321
Confusion Matrix:


,Predicted Negative,Predicted Positive
Actual Negative,297331,22082
Actual Positive,45697,465554


Loan Approved Distribution in Testing Set:
loan_approved
1    511251
0    319413
Name: count, dtype: int64


In [9]:
########################################
# Extracting Race, Gender, Ethnicity
########################################

test_results = test_df.copy()
test_results['Actual'] = y_test.values
test_results['Predicted'] = y_pred

race_reverse_mapping = {
    0: 'American Indian or Alaska Native',
    1: 'Asian',
    2: 'Black or African American',
    3: 'Native Hawaiian or Other Pacific Islander',
    4: 'White',
    5: 'Two or More Minority Races',
    6: 'Joint',
    7: 'Free Form Text Only',
    8: 'Race Not Available'
}

sex_reverse_mapping = {
    0: 'Male',
    1: 'Female',
    2: 'Joint',
    3: 'Sex Not Available'
}

ethnicity_reverse_mapping = {
    0: 'Hispanic or Latino',
    1: 'Not Hispanic or Latino',
    2: 'Joint',
    3: 'Ethnicity Not Available',
    4: 'Free Form Text Only'
}

def extract_group(row, prefix, reverse_mapping):
    one_hot_cols = [col for col in row.index if col.startswith(prefix)]
    for col in one_hot_cols:
        if row[col] == 1:
            try:
                value = int(col.split('_')[1])
                return reverse_mapping.get(value, "Unknown")
            except (IndexError, ValueError):
                return "Unknown"
    return f"{prefix.capitalize()} Not Available"

test_results['Race'] = test_results.apply(lambda row: extract_group(row, 'race', race_reverse_mapping), axis=1)
test_results['Gender'] = test_results.apply(lambda row: extract_group(row, 'gender', sex_reverse_mapping), axis=1)
test_results['Ethnicity'] = test_results.apply(lambda row: extract_group(row, 'ethnicity', ethnicity_reverse_mapping), axis=1)

print("\nUnique Races:", test_results['Race'].unique())
print("Unique Genders:", test_results['Gender'].unique())
print("Unique Ethnicities:", test_results['Ethnicity'].unique())
print("Unique SES Groups:", test_results['SES_group'].unique())


Unique Races: ['White' 'Race Not Available' 'Asian' 'Black or African American'
 'Native Hawaiian or Other Pacific Islander' 'Joint'
 'American Indian or Alaska Native' 'Two or More Minority Races'
 'Free Form Text Only']
Unique Genders: ['Male' 'Joint' 'Sex Not Available' 'Female']
Unique Ethnicities: ['Not Hispanic or Latino' 'Ethnicity Not Available' 'Hispanic or Latino'
 'Joint' 'Free Form Text Only']
Unique SES Groups: ['High' 'Low' 'Middle']


In [10]:
########################################
# Fairness Metric Functions
########################################

def statistical_parity(y_true, y_pred, group):
    return y_pred.groupby(group).mean()

def predictive_parity(y_true, y_pred, group):
    df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'group': group})
    precision = df.groupby('group', group_keys=False).apply(lambda x: precision_score(x['y_true'], x['y_pred'], zero_division=0))
    return precision

def true_positive_rate(y_true, y_pred, group):
    df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'group': group})
    tpr = df.groupby('group', group_keys=False).apply(lambda x: recall_score(x['y_true'], x['y_pred'], zero_division=0))
    return tpr

def false_positive_rate(y_true, y_pred, group):
    df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'group': group})
    fpr = {}
    for grp in df['group'].unique():
        grp_df = df[df['group'] == grp]
        if len(grp_df) == 0:
            fpr[grp] = 0
            continue
        cm = confusion_matrix(grp_df['y_true'], grp_df['y_pred'])
        if cm.shape == (1,1):
            if grp_df['y_true'].iloc[0] == 0:
                tn = cm[0,0]
                fp = 0
            else:
                tn = 0
                fp = cm[0,0]
        elif cm.shape == (2,2):
            tn, fp, fn, tp = cm.ravel()
        else:
            tn, fp, fn, tp = 0,0,0,0
        fpr[grp] = fp / (fp + tn) if (fp + tn) > 0 else 0
    return pd.Series(fpr)

def base_rate_preservation(y_true, group):
    return y_true.groupby(group).mean()

def compute_fairness_metrics(test_results, group_col):
    metrics = pd.DataFrame()
    metrics['Statistical Parity'] = statistical_parity(test_results['Actual'], test_results['Predicted'], test_results[group_col])
    metrics['Predictive Parity (Precision)'] = predictive_parity(test_results['Actual'], test_results['Predicted'], test_results[group_col])
    metrics['TPR (Sensitivity)'] = true_positive_rate(test_results['Actual'], test_results['Predicted'], test_results[group_col])
    metrics['FPR (Fallout)'] = false_positive_rate(test_results['Actual'], test_results['Predicted'], test_results[group_col])
    metrics['Base Rate'] = base_rate_preservation(test_results['Actual'], test_results[group_col])
    return metrics

def format_metrics(metrics_df):
    return metrics_df.round(2)

def create_fairness_table(metrics_df, group_type):
    table = metrics_df.reset_index()
    table = table.rename(columns={
        'index': group_type,
        'Statistical Parity': 'Statistical Parity',
        'Predictive Parity (Precision)': 'Predictive Parity',
        'TPR (Sensitivity)': 'TPR',
        'FPR (Fallout)': 'FPR',
        'Base Rate': 'Base Rate'
    })

    table['Statistical Parity'] = table['Statistical Parity'].apply(lambda x: f"{x:.2f}")
    table['Predictive Parity'] = table['Predictive Parity'].apply(lambda x: f"{x:.2f}")
    table['TPR'] = table['TPR'].apply(lambda x: f"{x:.2f}")
    table['FPR'] = table['FPR'].apply(lambda x: f"{x:.2f}")
    table['Base Rate'] = table['Base Rate'].apply(lambda x: f"{x:.2f}")
    return table

In [11]:
########################################
# Compute Fairness Metrics for Various Groups
########################################

fairness_race = compute_fairness_metrics(test_results, 'Race')
fairness_gender = compute_fairness_metrics(test_results, 'Gender')
fairness_ethnicity = compute_fairness_metrics(test_results, 'Ethnicity')
fairness_ses = compute_fairness_metrics(test_results, 'SES_group')

test_results['SES_Race'] = test_results['SES_group'] + ' SES × ' + test_results['Race']
fairness_ses_race = compute_fairness_metrics(test_results, 'SES_Race')

test_results['SES_Gender'] = test_results['SES_group'] + ' SES × ' + test_results['Gender']
fairness_ses_gender = compute_fairness_metrics(test_results, 'SES_Gender')

test_results['SES_Ethnicity'] = test_results['SES_group'] + ' SES × ' + test_results['Ethnicity']
fairness_ses_ethnicity = compute_fairness_metrics(test_results, 'SES_Ethnicity')

# Create fully intersectional column (SES × Race × Gender × Ethnicity)
test_results['SES_Race_Gender_Ethnicity'] = (
    test_results['SES_group'] + ' × ' +
    test_results['Race'] + ' × ' +
    test_results['Gender'] + ' × ' +
    test_results['Ethnicity']
)

fairness_full_intersection = compute_fairness_metrics(test_results, 'SES_Race_Gender_Ethnicity')

# Also create a Race × Gender × Ethnicity without SES
test_results['Race_Gender_Ethnicity'] = (
    test_results['Race'] + ' × ' +
    test_results['Gender'] + ' × ' +
    test_results['Ethnicity']
)
fairness_non_ses = compute_fairness_metrics(test_results, 'Race_Gender_Ethnicity')

<ipython-input-10-e1b3dd38566a>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  precision = df.groupby('group', group_keys=False).apply(lambda x: precision_score(x['y_true'], x['y_pred'], zero_division=0))
<ipython-input-10-e1b3dd38566a>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tpr = df.groupby('group', group_keys=False).apply(lambda x: recall_score(x['y_true'], x['y_pred'], zero_division=0))
<ip

In [12]:
########################################
# Additional Formatting for Intersectional Tables
########################################

def create_intersectional_table(metrics_df, column_name):
    table = metrics_df.reset_index()
    table = table.rename(columns={'index': 'Intersection'})
    # Intersection format: "Low × Black or African American × Female × Hispanic or Latino"
    parts = table['Intersection'].str.split(' × ', expand=True)
    if parts.shape[1] == 4:
        # Full intersection (SES, Race, Gender, Ethnicity)
        table[['SES', 'Race', 'Gender', 'Ethnicity']] = parts
        table.drop(columns=['Intersection'], inplace=True)
        columns_order = ['SES', 'Race', 'Gender', 'Ethnicity', 'Statistical Parity', 'Predictive Parity (Precision)', 'TPR (Sensitivity)', 'FPR (Fallout)', 'Base Rate']
    elif parts.shape[1] == 3:
        # Race × Gender × Ethnicity
        table[['Race', 'Gender', 'Ethnicity']] = parts
        table.drop(columns=['Intersection'], inplace=True)
        columns_order = ['Race', 'Gender', 'Ethnicity', 'Statistical Parity', 'Predictive Parity (Precision)', 'TPR (Sensitivity)', 'FPR (Fallout)', 'Base Rate']
    else:
        # Default to no splitting if unexpected
        columns_order = table.columns

    table = table[columns_order]

    # Format numeric columns
    numeric_cols = ['Statistical Parity', 'Predictive Parity (Precision)', 'TPR (Sensitivity)', 'FPR (Fallout)', 'Base Rate']
    for col in numeric_cols:
        if col in table.columns:
            table[col] = table[col].apply(lambda x: f"{x:.2f}")
    return table

In [13]:
########################################
# Display Results
########################################

# Basic group tables
print("\n### Fairness Metrics by Race ###\n")
display(create_fairness_table(format_metrics(fairness_race), 'Race'))

print("\n### Fairness Metrics by Gender ###\n")
display(create_fairness_table(format_metrics(fairness_gender), 'Gender'))

print("\n### Fairness Metrics by Ethnicity ###\n")
display(create_fairness_table(format_metrics(fairness_ethnicity), 'Ethnicity'))

print("\n### Fairness Metrics by SES Group ###\n")
display(create_fairness_table(format_metrics(fairness_ses), 'SES Group'))

print("\n### Fairness Metrics by SES × Race ###\n")
display(create_fairness_table(format_metrics(fairness_ses_race), 'Intersection (SES × Race)'))

print("\n### Fairness Metrics by SES × Gender ###\n")
display(create_fairness_table(format_metrics(fairness_ses_gender), 'Intersection (SES × Gender)'))

print("\n### Fairness Metrics by SES × Ethnicity ###\n")
display(create_fairness_table(format_metrics(fairness_ses_ethnicity), 'Intersection (SES × Ethnicity)'))

# Full intersection table (SES × Race × Gender × Ethnicity)
print("\n### Fairness Metrics by SES × Race × Gender × Ethnicity ###\n")
display(create_intersectional_table(format_metrics(fairness_full_intersection), 'SES_Race_Gender_Ethnicity'))

# Non-SES intersection (Race × Gender × Ethnicity)
print("\n### Fairness Metrics by Race × Gender × Ethnicity ###\n")
display(create_intersectional_table(format_metrics(fairness_non_ses), 'Race_Gender_Ethnicity'))

# Re-evaluate the model under a different name if needed
evaluate_model(y_test, y_pred, "model_4")


### Fairness Metrics by Race ###



,Race,Statistical Parity,Predictive Parity,TPR,FPR,Base Rate
0,American Indian or Alaska Native,0.42,0.95,0.87,0.04,0.45
1,Asian,0.54,0.95,0.89,0.06,0.58
2,Black or African American,0.45,0.95,0.88,0.05,0.48
3,Free Form Text Only,0.29,0.96,0.81,0.02,0.34
4,Joint,0.63,0.96,0.92,0.08,0.65
5,Native Hawaiian or Other Pacific Islander,0.42,0.95,0.86,0.04,0.46
6,Race Not Available,0.66,0.95,0.93,0.09,0.67
7,Two or More Minority Races,0.39,0.96,0.86,0.03,0.44
8,White,0.57,0.96,0.90,0.06,0.60



### Fairness Metrics by Gender ###



,Gender,Statistical Parity,Predictive Parity,TPR,FPR,Base Rate
0,Female,0.51,0.95,0.89,0.06,0.55
1,Joint,0.60,0.96,0.90,0.06,0.64
2,Male,0.50,0.94,0.89,0.06,0.53
3,Sex Not Available,0.76,0.96,0.95,0.13,0.76



### Fairness Metrics by Ethnicity ###



...

,SES_Gender,Statistical Parity,Predictive Parity,TPR,FPR,Base Rate
0,High SES × Female,0.54,0.95,0.89,0.06,0.57
1,High SES × Joint,0.62,0.97,0.91,0.06,0.66
2,High SES × Male,0.53,0.95,0.90,0.06,0.55
3,High SES × Sex Not Available,0.77,0.96,0.96,0.13,0.78
4,Low SES × Female,0.48,0.94,0.88,0.06,0.51
5,Low SES × Joint,0.56,0.95,0.90,0.06,0.59
6,Low SES × Male,0.46,0.94,0.89,0.06,0.49
7,Low SES × Sex Not Available,0.74,0.95,0.95,0.14,0.74
8,Middle SES × Female,0.52,0.95,0.89,0.06,0.56
9,Middle SES × Joint,0.60,0.96,0.90,0.07,0.64



### Fairness Metrics by SES × Ethnicity ###



,SES_Ethnicity,Statistical Parity,Predictive Parity,TPR,FPR,Base Rate
0,High SES × Ethnicity Not Available,0.70,0.96,0.94,0.10,0.72
1,High SES × Free Form Text Only,0.37,0.93,0.87,0.04,0.39
2,High SES × Hispanic or Latino,0.51,0.95,0.89,0.06,0.55
3,High SES × Joint,0.61,0.96,0.92,0.06,0.64
4,High SES × Not Hispanic or Latino,0.58,0.96,0.90,0.06,0.62
5,Low SES × Ethnicity Not Available,0.67,0.95,0.93,0.11,0.68
6,Low SES × Free Form Text Only,0.38,0.96,0.89,0.02,0.41
7,Low SES × Hispanic or Latino,0.46,0.94,0.89,0.06,0.49
8,Low SES × Joint,0.55,0.94,0.92,0.07,0.57
9,Low SES × Not Hispanic or Latino,0.52,0.95,0.90,0.06,0.55



### Fairness Metrics by SES × Race × Gender × Ethnicity ###



KeyError: 'Intersection'